In [1]:
!pip install utils

import torch
import torch.nn as nn
import torch.utils.checkpoint as checkpoint
from timm.models.layers import DropPath, to_2tuple, trunc_normal_
import torch.nn.functional as F
from einops import rearrange, repeat
from einops.layers.torch import Rearrange
import math
import numpy as np
import time
from torch import einsum
import cv2
import scipy.misc
import utils

  Preparing metadata (setup.py) ... done
  Created wheel for utils: filename=utils-1.0.2-py2.py3-none-any.whl size=13906 sha256=973d90742ccdb63f21c95b2d1405cbdde4607c9b874ac0455134aaa528064d47
  Stored in directory: /root/.cache/pip/wheels/15/0c/b3/674aea8c5d91c642c817d4d630bd58faa316724b136844094d
Successfully built utils


/usr/local/lib/python3.11/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


In [2]:
from google.colab import drive
import pickle

drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
with open('./drive/MyDrive/data/V_resize_224.pkl', 'rb') as f:
     resize_224 = pickle.load(f)
     print('V resize 224 data : ', len(resize_224))

V resize 224 data :  4000


In [4]:
X = []
y = []
for file_name, data in resize_224.items():
    X.append(data["matrix"])
    y.append(data["pothole"])

X = np.array(X)
y = np.array(y)

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

print("Training set size:", len(X_train))
print("Validation set size:", len(X_val))
print("Testing set size:", len(X_test))

print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

Training set size: 3200
Validation set size: 400
Testing set size: 400
(3200, 224, 224)
(400, 224, 224)
(400, 224, 224)


In [8]:
# prompt: change X_train, X_val, X_test to 1 channel

X_train = np.expand_dims(X_train, axis=1)
X_val = np.expand_dims(X_val, axis=1)
X_test = np.expand_dims(X_test, axis=1)

print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

y_train = np.array(y_train)
y_val = np.array(y_val)
y_test = np.array(y_test)
y_train = np.expand_dims(y_train, axis=1)
y_val = np.expand_dims(y_val, axis=1)
y_test = np.expand_dims(y_test, axis=1)


(3200, 1, 1, 224, 224)
(400, 1, 1, 224, 224)
(400, 1, 1, 224, 224)


In [9]:
# prompt: permute the train/val/test to have channel in front

print(X_train.shape)
print(X_val.shape)
print(X_test.shape)
print(y_train.shape)
print(y_val.shape)
print(y_test.shape)

(3200, 1, 1, 224, 224)
(400, 1, 1, 224, 224)
(400, 1, 1, 224, 224)
(3200, 1)
(400, 1)
(400, 1)


In [10]:
train_dataset = torch.utils.data.TensorDataset(torch.from_numpy(X_train).float(), torch.from_numpy(y_train).long())
val_dataset = torch.utils.data.TensorDataset(torch.from_numpy(X_val).float(), torch.from_numpy(y_val).long())
test_dataset = torch.utils.data.TensorDataset(torch.from_numpy(X_test).float(), torch.from_numpy(y_test).long())

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)

In [11]:
import torchvision.models as models

class ResNet18Binary(nn.Module):
    def __init__(self, num_classes=2):
        super(ResNet18Binary, self).__init__()

        self.resnet18 = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)

        # Modify the first convolutional layer to accept 1-channel input
        self.resnet18.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)

        # Modify the last fully connected layer for binary classification
        self.resnet18.fc = nn.Linear(self.resnet18.fc.in_features, 2)

    def forward(self, x):
        return self.resnet18(x)

In [15]:
model = ResNet18Binary()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [18]:
num_epochs = 30

train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []

In [19]:
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    train_correct = 0
    train_total = 0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        # Remove the extra dimension using squeeze
        inputs = inputs.squeeze(2)  # Remove dimension at index 2
        outputs = model(inputs)
        loss = criterion(outputs, labels.squeeze(1)) # Remove extra dimension from labels
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        train_total += labels.size(0)
        train_correct += (predicted == labels.squeeze(1)).sum().item() # Remove extra dimension from labels
        train_losses.append(loss.item())
        train_accuracies.append(100 * train_correct / train_total)

    # Validation loop
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            # Remove the extra dimension using squeeze
            inputs = inputs.squeeze(2)  # Remove dimension at index 2
            outputs = model(inputs)
            loss = criterion(outputs, labels.squeeze(1))
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels.squeeze(1)).sum().item() # Remove extra dimension from labels
            val_losses.append(loss.item())
            val_accuracies.append(100 * val_correct / val_total)

    print(f'Epoch [{epoch+1}/{num_epochs}], '
          f'Train Loss: {train_loss / len(train_loader):.4f}, Train Acc: {100 * train_correct / train_total:.2f}%, '
          f'Val Loss: {val_loss / len(val_loader):.4f}, Val Acc: {100 * val_correct / val_total:.2f}%')

# Testing loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        # Remove the extra dimension using squeeze
        inputs = inputs.squeeze(2)  # Remove dimension at index 2
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels.squeeze(1)).sum().item()  # Remove extra dimension from labels

print(f'Test Accuracy: {100 * correct / total:.2f}%')

Epoch [1/30], Train Loss: 0.6006, Train Acc: 68.50%, Val Loss: 0.7579, Val Acc: 74.00%
Epoch [2/30], Train Loss: 0.4854, Train Acc: 76.78%, Val Loss: 0.5796, Val Acc: 64.75%
Epoch [3/30], Train Loss: 0.4326, Train Acc: 79.50%, Val Loss: 0.4284, Val Acc: 79.25%
Epoch [4/30], Train Loss: 0.3442, Train Acc: 84.97%, Val Loss: 0.3912, Val Acc: 82.50%
Epoch [5/30], Train Loss: 0.3060, Train Acc: 86.31%, Val Loss: 0.4004, Val Acc: 83.75%
Epoch [6/30], Train Loss: 0.2787, Train Acc: 88.19%, Val Loss: 0.3206, Val Acc: 88.75%
Epoch [7/30], Train Loss: 0.2303, Train Acc: 90.56%, Val Loss: 0.3148, Val Acc: 89.25%
Epoch [8/30], Train Loss: 0.1863, Train Acc: 92.44%, Val Loss: 0.3483, Val Acc: 86.75%
Epoch [9/30], Train Loss: 0.1751, Train Acc: 92.97%, Val Loss: 0.4277, Val Acc: 85.00%
Epoch [10/30], Train Loss: 0.1555, Train Acc: 93.53%, Val Loss: 0.3215, Val Acc: 88.00%
Epoch [11/30], Train Loss: 0.1489, Train Acc: 94.25%, Val Loss: 0.3424, Val Acc: 88.25%
Epoch [12/30], Train Loss: 0.1272, Train 